In [1]:
import pandas as pd
import os
import numpy as np
# from sksurv.metrics import concordance_index_censored

In [2]:
from pycox.evaluation.concordance import concordance_td
from pycox.evaluation import EvalSurv

In [3]:
#!pip install -qU scikit_survival 

In [4]:
#!pip install -qU pycox 

In [5]:
#!pip install -qU torch 

In [6]:
os.getcwd()

'/home/ronin/Dev/miniconda3/notebooks/thesis_notebook'

In [7]:
#pth = "zenodo/metrics/TCGA/BLCA/early/clinical_gex/"
#pthsf = "zenodo/survival_functions/TCGA/BLCA/early/clinical_gex/"
metrics_csv = "metrics/BLCA/intermediate_mean/clinical_gex/metrics.csv"
data_csv = "processed/BLCA_data_preprocessed.csv"
splits_test_csv = "splits/BLCA_test_splits.csv"
splits_train_csv = "splits/BLCA_train_splits.csv"
survival_funct_path = "survival_functions/BLCA/late_mean/clinical_gex/" ## + split_1.csv etc

In [8]:
#pd.read_csv(pth + "metrics.csv");
pd.read_csv(metrics_csv);

In [12]:
#split = pd.read_csv(pthsf + "split_1.csv")
splits_test = pd.read_csv(splits_test_csv)
splits_test.iloc[:10,:10]

,0,1,2,3,4,5,6,7,8,9
0,0,8,12,13,14,22,23,25,29,33
1,1,3,5,9,15,24,27,30,31,38
2,6,10,17,20,28,42,46,52,54,60
3,11,16,18,26,32,35,41,47,48,51
4,2,4,7,19,21,37,40,45,50,55
5,11,12,15,27,32,45,46,61,63,67
6,8,9,10,17,18,19,21,35,40,44
7,2,3,5,6,7,13,14,20,28,33
8,0,1,22,23,25,26,29,30,31,34
9,4,16,24,37,39,41,52,53,64,65


In [14]:
#concordance_index_censored(event_indicator=)

In [14]:
X = pd.read_csv(data_csv)

In [15]:
X.iloc[:10, :10]

,patient_id,OS,OS_days,clinical_age_at_initial_pathologic_diagnosis,clinical_gender,clinical_race,clinical_ajcc_pathologic_tumor_stage,clinical_clinical_stage,clinical_histological_type,cnv_ACAP3
0,TCGA-ZF-AA5P,0,372,65,MALE,WHITE,Stage IV,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,0
1,TCGA-ZF-AA58,0,1649,61,FEMALE,WHITE,Stage IV,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,0
2,TCGA-ZF-AA56,1,259,79,FEMALE,WHITE,Stage III,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,0
3,TCGA-ZF-AA54,1,590,71,MALE,WHITE,Stage III,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,-1
4,TCGA-ZF-AA53,0,1761,60,MALE,WHITE,Stage II,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,1
5,TCGA-ZF-AA52,1,1077,70,MALE,WHITE,Stage III,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,1
6,TCGA-ZF-AA51,0,1714,69,FEMALE,WHITE,Stage II,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,1
7,TCGA-ZF-AA4W,0,1830,56,MALE,WHITE,Stage III,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,0
8,TCGA-ZF-AA4V,0,1806,66,MALE,WHITE,Stage III,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,0
9,TCGA-ZF-AA4T,1,599,65,MALE,WHITE,Stage IV,NaN,Muscle invasive urothelial carcinoma (pT2 or a...,1


In [23]:
event_indicator, event_time = X["OS"], X["OS_days"]

In [26]:
event_time[:10]
#event_indicator[:10]

0     372
1    1649
2     259
3     590
4    1761
5    1077
6    1714
7    1830
8    1806
9     599
Name: OS_days, dtype: int64

In [20]:
#split.cumprod(axis=1).sum(axis=1)

In [25]:
pd.Series(['0','1']).is_monotonic_increasing


True

In [ ]:
l = []
##prova
for idx in range(25):
    split = pd.read_csv(pthsf + f"split_{idx+1}.csv")
    estimate = split.cumprod(axis=1).sum(axis=1)
    
    test_idx = pd.read_csv('zenodo/splits/BLCA_test_splits.csv').iloc[idx, :].tolist()
    
    #cidx = concordance_index_censored(
    #    event_indicator=event_indicator[test_idx] == 1,
    #    event_time = event_time[test_idx],
    #    estimate=estimate
    #)
    cidx = concordance_td(
        surv= sur
        durations=event_time.to_numpy(),
        events=event_indicator.to_numpy(),
        surv_idx=split.to_numpy().T,
    )
    
    l.append(cidx)

NameError: name 'pthsf' is not defined

In [ ]:
pd.Series(split.to_numpy().T).is_monotonic

In [ ]:
[el[0] for el in l]

In [ ]:
event_time.size

In [ ]:
(event_time.unique()).size

In [ ]:
split.columns

In [ ]:
## dal valore

In [28]:
def create_surv_idx(splits, survival_function, data):
    """
    Arguments:
    splits {np.array[n_splits+1]} -- Array of split points defining the durations.
    survival_function {np.array[n_splits, n_individuals]} -- Survival function.
    data {np.array[n_individuals]} -- Array of event times or censoring times.
    
    Returns:
    np.array[n_individuals] -- The 'surv_idx' array mapping event times to survival function indices.
    """
    surv_idx = np.zeros(len(data), dtype=int)
    for i, event_time in enumerate(X):
        # Find the index of the split that the event time belongs to
        split_index = np.searchsorted(splits, event_time)
        # Assign the corresponding survival function index
        surv_idx[i] = split_index - 1
    
    return surv_idx

In [29]:
create_surv_idx()

TypeError: create_surv_idx() missing 3 required positional arguments: 'splits', 'survival_function', and 'data'

In [68]:
l = []
for idx in range(25):
    split = pd.read_csv( survival_funct_path + f"split_{idx+1}.csv")
    estimate = split.cumprod(axis=1).sum(axis=1)
    test_idx = pd.read_csv(split_test_path).iloc[idx, :].tolist()
    
    
    split_event_time = event_time[test_idx].to_numpy()
    split_event_indicator = event_indicator[test_idx].to_numpy()
    surv = split.to_numpy().T #lo vuole lui così
    
    
    cidx = concordance_td(
        surv=surv,
        durations=split_event_time,
        events=split_event_indicator,
        surv_idx=np.searchsorted(np.sort(np.unique(split_event_time)), split_event_time),)
    l.append(cidx)

In [69]:
l;

In [70]:
(np.array(l) - pd.read_csv("metrics/BLCA/intermediate_mean/clinical_gex/metrics.csv")["concordance"].to_numpy())

array([-0.00460829, -0.00648749, -0.10616153, -0.04981203, -0.07249216,
        0.02951389, -0.03325123, -0.05272256, -0.01949861,  0.01580334,
        0.02709568, -0.04029991, -0.00448029, -0.01565217, -0.0235492 ,
        0.01911381, -0.03071017,  0.03649001,  0.03876582, -0.02982293,
       -0.02932255,  0.00429185, -0.02461034, -0.01904762, -0.01226994])

In [ ]:
l = []
####!!!!!!!
for idx in range(25):
    split = pd.read_csv(pthsf + f"split_{idx+1}.csv")
    estimate = split.cumprod(axis=1).sum(axis=1)
    
    test_idx = pd.read_csv('zenodo/splits/BLCA_test_splits.csv').iloc[idx, :].tolist()
    
    #cidx = concordance_index_censored(
    #    event_indicator=event_indicator[test_idx] == 1,
    #    event_time = event_time[test_idx],
    #    estimate=estimate
    #)
    split_event_time = event_time[test_idx].to_numpy()
    split_event_indicator = event_indicator[test_idx].to_numpy()
    surv = split.to_numpy().T
    
    cidx = EvalSurv(
        surv=split.T,
        durations=split_event_time,
        events=split_event_indicator
    ).concordance_td()
  #  cidx = concordance_td(
   #     surv=surv,
    #    durations=split_event_time,
     #   events=split_event_indicator,
       # surv_idx=np.searchsorted(np.sort(np.unique(split_event_time)), split_event_time),
    #)
    
    l.append(cidx)